In [1]:
import json
import collections
from collections import defaultdict
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from collections import Counter
import math
from statistics import mean

import numpy as np
from numpy import dot
from numpy.linalg import norm

import scipy
from scipy.optimize import linear_sum_assignment

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/anyaji/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [4]:
from nltk.tokenize import word_tokenize as tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [5]:
f = open('../dense_final3.json')
data = json.load(f)

In [6]:
def clean(x):
    words = tokenize(x.lower())
    wl = [stemmer.stem(w) for w in words if w not in stop_words and (w.islower() or w.isalnum())]
    return wl

# Stats

In [7]:
def print_stats(a):
    print(np.mean(np.array(a)), np.std(np.array(a)))
def desc_len(a):
    return len(re.findall(r'\w+', a))

In [8]:
#Length - whole
wholeanns_lengths=[]

for worker, detail in data['users'].items():
    for k, v in detail.items():
        if k.startswith('page'):
            text=v['whole-annotation']['wholeAnnotation']
            wholeanns_lengths.append(desc_len(text))

print_stats(wholeanns_lengths)
print(len(wholeanns_lengths))

2.2799910474485228 1.6192799516580405
13404


In [15]:
#Length - part
partanns_lengths=[]
for worker, detail in data['users'].items():
    for k, v in detail.items():
        if k.startswith('page'):
            parts_set = set(v['piece-annotation'].values()) # parts, excluding duplicates
            for part_ann in parts_set:
                partanns_lengths.append(desc_len(part_ann))
                
print_stats(partanns_lengths)


1.3088673698061226 0.7734742484068301


In [16]:
#Parts per shape
nums_parts=[]
for worker, detail in data['users'].items():
    for k, v in detail.items():
        if k.startswith('page'):
            c = len(set(v['piece-annotation'].values()))
            nums_parts.append(c)
print_stats(nums_parts)


3.628692927484333 1.2844188468346154


In [17]:
#Pieces per part
def count_ppp(d):
    '''returns ppp for all parts in this tangram'''
    partdesc_to_count=defaultdict(int)
    for idx, ann in d.items():
        partdesc_to_count[ann]+=1
    return list(partdesc_to_count.values())

ppp=[]
for worker, detail in data['users'].items():
    for k, v in detail.items():
        if k.startswith('page'):
            ppp+=count_ppp(v['piece-annotation'])
print_stats(ppp)

1.9290692654043051 1.2027307156975353


In [19]:
def clean_vocab(x):
    words = tokenize(x.lower())
    wl = [stemmer.stem(w) for w in words if (w.islower() or w.isalnum())] 
    return wl

In [21]:
#Whole vocab
whole_vocab=set()
for worker, detail in data['users'].items():
    for k, v in detail.items():
        if k.startswith('page'):
            text=clean_vocab(v['whole-annotation']['wholeAnnotation'])
            for w in text:
                whole_vocab.add(w)
print(len(whole_vocab))

3031


In [22]:
#Part vocab
part_vocab=set()
for worker, detail in data['users'].items():
    for k, v in detail.items():
        if k.startswith('page'):
            piece_anns = set(v['piece-annotation'].values())
            for ann in piece_anns:
                text=clean_vocab(ann)
                for w in text:
                    part_vocab.add(w)
print(len(part_vocab))

3110


In [23]:
len(set.union(whole_vocab, part_vocab))

4522